In [35]:
'''
Make Pseudo tempature LAS logs from corrected bottom hole tempatures.

Copyright (c) 2023 Matthew W. Bauer, PG

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
'''

import pandas as pd
import lasio
import numpy as np

In [36]:
## Dumby Data
wells = pd.DataFrame({'API':['0123456789'],
                     'BHTcorr':[180],
                     'BHTdepth':[2000],
                      'STRT':[0],
                     'SurfAvg':[55]})
#Real Data
#wells = pd.read_excel('your_file.xlsx')

wells.head()

,API,BHTcorr,BHTdepth,STRT,SurfAvg
0,0123456789,180,2000,0,55


In [39]:
stepdepth = 1
depthunits = 'ft'
tempunits = 'degF'

for index, row in wells.iterrows():
    #break
    n = int(row['BHTdepth'] / stepdepth)
    tempCurve = np.linspace(row['SurfAvg'],row['BHTcorr'],num=n)
    DEPT = np.linspace(row['STRT'],row['BHTdepth'],num=n)

    df = pd.DataFrame({'DEPT':DEPT,'TEMP':tempCurve})
    df.DEPT = df.DEPT.round(1)
    df.TEMP = df.TEMP.round(3)

    las = lasio.LASFile()
    las.well.API['value'] = row['API']
    las.well.STRT = lasio.HeaderItem('STRT', value=row['STRT'], unit=depthunits)
    las.well.STOP = lasio.HeaderItem('STOP', value=row['BHTdepth'], unit=depthunits)
    las.well.STEP = lasio.HeaderItem('STEP', value=stepdepth, unit=depthunits)
    las.well['COMP'] = lasio.HeaderItem('COMP', value='TempLASpy, by MWB')

    # Other columns
    other_cols = ['NULL','WELL','FLD','LOC','PROV','CNTY','STAT','CTRY','SRVC','DATE','UWI']

    for col in other_cols:
        if col in list(wells):
            if str(row[col]) != 'nan':
                las.well[col] = lasio.HeaderItem(col, value=row[col])


    #las.params['DEPT'] = lasio.HeaderItem('DEPT', value=depthunits)
    #las.params['TEMP'] = lasio.HeaderItem('TEMP', value=tempunits)

    las.append_curve('DEPT', df.DEPT.values, unit=depthunits)
    las.append_curve('DEPT', df.TEMP.values, unit=tempunits)

    filename = row['API'] + '_TempLAS.las'
    las.write(filename, version=1.2)